In [17]:
import torch.nn as nn
import torch as tch
import numpy as np, pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score,roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

# Load data into memory using pandas
$ X = \{(x_1, y_i),\; (x_2, y_i),\; ...,\;(x_{11162}, y_i)\} ,\qquad  where\:  X \in \mathbb{R}^{11162} \quad, \quad and \;i = \{1,2\} \; and \:y_i = \{0,1\} $ <br><br>
* The features $x_i$ take diffrents types of values (`strings`, `int`, `binary values`, ....).
* We should rectifie this, in order to transforme this probleme to a binary classification probleme with $\;y_i\in \{0,1\}$.

In [18]:
df = pd.read_csv("./bank.csv")
print("DF Shape : ", df.shape)
df.head()

DF Shape :  (11162, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


# Distributing the Target Values

In [19]:
print("Distribution of Target Values in Dataset :")
df.deposit.value_counts()

Distribution of Target Values in Dataset :


no     5873
yes    5289
Name: deposit, dtype: int64

count the number of occurrences of each unique value in the `deposit` column of a DataFrame `df`.

# Distributing the NA (Null) Values in the Dataset

* Check if we have `na` values within the dataset

In [20]:
df.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

# Check the distinct datatypes within the dataset

In [21]:
df.dtypes.value_counts()

object    10
int64      7
dtype: int64

# Extract categorical columns from datase

In [22]:
categorical_columns = df.select_dtypes(include="object").columns
print("Categorical cols:",list(categorical_columns))

Categorical cols: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']


In [23]:
# For each categorical column if values in (Yes/No) convert into a 1/0 flag
for col in categorical_columns:
    if df[col].nunique() == 2:
        df[col] = np.where(df[col] == "yes", 1, 0)

In [24]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,0,2343,1,0,unknown,5,may,1042,1,-1,0,unknown,1
1,56,admin.,married,secondary,0,45,0,0,unknown,5,may,1467,1,-1,0,unknown,1
2,41,technician,married,secondary,0,1270,1,0,unknown,5,may,1389,1,-1,0,unknown,1
3,55,services,married,secondary,0,2476,1,0,unknown,5,may,579,1,-1,0,unknown,1
4,54,admin.,married,tertiary,0,184,0,0,unknown,5,may,673,2,-1,0,unknown,1


# One-Hot Encoding for the Remaining Non-Binary Categorical Variables
* For the remaining cateogrical variables;
* create one-hot encoded version of the dataset

In [25]:
new_df = pd.get_dummies(df)
print(new_df.shape)
new_df.head()

(11162, 49)


,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,59,0,2343,1,0,5,1042,1,-1,0,...,0,0,1,0,0,0,0,0,0,1
1,56,0,45,0,0,5,1467,1,-1,0,...,0,0,1,0,0,0,0,0,0,1
2,41,0,1270,1,0,5,1389,1,-1,0,...,0,0,1,0,0,0,0,0,0,1
3,55,0,2476,1,0,5,579,1,-1,0,...,0,0,1,0,0,0,0,0,0,1
4,54,0,184,0,0,5,673,2,-1,0,...,0,0,1,0,0,0,0,0,0,1


* Define target and predictors for the model

In [26]:
target = "deposit"
predictors = set(new_df.columns) - set([target])
print("new_df.shape:",new_df.shape)
new_df[predictors].head()

new_df.shape: (11162, 49)


,month_dec,month_sep,default,poutcome_success,campaign,age,month_apr,month_oct,marital_single,month_mar,...,poutcome_failure,month_jan,contact_telephone,day,marital_married,job_housemaid,previous,month_jun,education_secondary,contact_cellular
0,0,0,0,0,1,59,0,0,0,0,...,0,0,0,5,1,0,0,0,1,0
1,0,0,0,0,1,56,0,0,0,0,...,0,0,0,5,1,0,0,0,1,0
2,0,0,0,0,1,41,0,0,0,0,...,0,0,0,5,1,0,0,0,1,0
3,0,0,0,0,1,55,0,0,0,0,...,0,0,0,5,1,0,0,0,1,0
4,0,0,0,0,2,54,0,0,0,0,...,0,0,0,5,1,0,0,0,0,0


# Preparing the Dataset for Training and Validation

* Convert all datatypes within pandas dataframe to Float32
* (Compatibility with PyTorch tensors)

In [27]:
new_df = new_df.astype(np.float32)

# Split dataset into Train/Test [80:20]


In [28]:
X_train,x_test, Y_train,y_test = train_test_split(new_df[predictors],new_df[target],test_size= 0.2)

# Convert Pandas dataframe, first to numpy and then to Torch Tensors

In [29]:
X_train = tch.from_numpy(X_train.values)
x_test = tch.from_numpy(x_test.values)
Y_train = tch.from_numpy(Y_train.values).reshape(-1,1)
y_test = tch.from_numpy(y_test.values).reshape(-1,1)

# Print the dataset size to verify

In [30]:
print("X_train.shape:",X_train.shape)
print("x_test.shape:",x_test.shape)
print("Y_train.shape:",Y_train.shape)
print("y_test.shape:",y_test.shape)

X_train.shape: torch.Size([8929, 48])
x_test.shape: torch.Size([2233, 48])
Y_train.shape: torch.Size([8929, 1])
y_test.shape: torch.Size([2233, 1])


# Define function to train the network
